# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../API HW/Sorted_Cities.csv")
df

,Unnamed: 0,City,Latitude,Longitude,Humidity,Pressure,Wind Speed (Mph),Cloud Cover (%),Temperature (F)
0,0,vila franca do campo,37.72,-25.43,93,1015,19.46,75,64.40
1,1,bredasdorp,-34.53,20.04,81,1020,4.70,72,51.80
2,2,codrington,-38.27,141.97,55,1002,32.37,100,53.67
3,3,sao joao da barra,-21.64,-41.05,86,1017,7.45,100,71.76
4,4,roebourne,-20.78,117.13,18,1016,20.45,0,81.39
...,...,...,...,...,...,...,...,...,...
569,569,carauari,-4.88,-66.90,94,1014,1.50,100,74.48
570,570,matara,5.95,80.54,68,1012,8.46,18,87.17
571,571,victoria,22.29,114.16,69,1010,8.01,91,79.52
572,572,polunochnoye,60.87,60.42,82,1008,5.79,91,34.61


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations1 = df[["Latitude", "Longitude"]]

fig = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(locations1, weights= df.Humidity, dissipating=False, max_intensity=100,
                                point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#My ideal weather
temp_mask = df.loc[(df['Temperature (F)'] >= 70) & (df['Temperature (F)']<= 85)]
wind_mask = df.loc[df['Wind Speed (Mph)'] < 10]
cloud_mask = df.loc[df['Cloud Cover (%)'] == 0]

#Combined df's
merge_1 = pd.merge (temp_mask,wind_mask, copy=False)
ideal_weather = pd.merge(merge_1,cloud_mask, copy=False).drop('Unnamed: 0',axis=1)
ideal_weather

,City,Latitude,Longitude,Humidity,Pressure,Wind Speed (Mph),Cloud Cover (%),Temperature (F)
0,san patricio,19.22,-104.70,78,1014,3.69,0,71.01
1,pochutla,15.74,-96.47,62,1011,3.36,0,82.99
2,ixtapa,20.70,-105.20,61,1014,1.48,0,71.87
3,la reforma,25.08,-108.05,83,1007,2.84,0,73.94
4,harper,4.38,-7.72,83,1012,2.30,0,79.81
5,namibe,-15.20,12.15,60,1012,3.02,0,76.91
6,arandis,-22.42,14.97,23,1016,8.21,0,71.62
7,paita,-5.09,-81.11,76,1014,7.61,0,71.69
8,maumere,-8.62,122.21,63,1009,1.70,0,84.90


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = ideal_weather
hotel_df['Hotel Name'] = ""
hotel_df['Hotel Address'] = ""
hotel_df['Hotel Rating']= ""
hotel_df

,City,Latitude,Longitude,Humidity,Pressure,Wind Speed (Mph),Cloud Cover (%),Temperature (F),Hotel Name,Hotel Address,Hotel Rating
0,san patricio,19.22,-104.70,78,1014,3.69,0,71.01,,,
1,pochutla,15.74,-96.47,62,1011,3.36,0,82.99,,,
2,ixtapa,20.70,-105.20,61,1014,1.48,0,71.87,,,
3,la reforma,25.08,-108.05,83,1007,2.84,0,73.94,,,
4,harper,4.38,-7.72,83,1012,2.30,0,79.81,,,
5,namibe,-15.20,12.15,60,1012,3.02,0,76.91,,,
6,arandis,-22.42,14.97,23,1016,8.21,0,71.62,,,
7,paita,-5.09,-81.11,76,1014,7.61,0,71.69,,,
8,maumere,-8.62,122.21,63,1009,1.70,0,84.90,,,


In [6]:
#Loop to find hotels within 5000 meters of Lat/Long coordinates, save 1st hotel.
for index, row in hotel_df.iterrows():
    
    #Concatinate coordinates for parameter. Cannot concat ints must convert to str().
    target_Loc = str(row.Latitude) + ", " + str(row.Longitude)
    
    #base google map api for location queries
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    #parameters used in url.
    params1 = {
    "location": target_Loc,
    "keyword": "Hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key}
    
    #Call API URL + parameters, convert to json
    response = requests.get(base_url, params=params1).json()
    
    #Locate keys for variables
    results = response['results']
    
    #Append results to hotel_df
    hotel_df.loc[index,'Hotel Name'] = results[0]['name']
    hotel_df.loc[index,'Hotel Address'] = results[0]['plus_code']['compound_code']
    hotel_df.loc[index,'Hotel Rating'] = results[0]['rating']

In [6]:
countries = []
#Create function to locate country data from geocode API using reverse geocode lookup.
def pullCountryforcoord(lat, long):
    
    #Pulls coordinate data by index from hotel_df
    lat = hotel_df["Latitude"][index]
    lng = hotel_df["Longitude"][index]
    
    #Google Geocode API for calls
    r_geo = "https://maps.googleapis.com/maps/api/geocode/json?"
    
    #concat latitude and longitude for parameter
    params2 = {"latlng": f"{lat},{lng}",
                 "key": g_key}
    
    #Call API URL + parameters, convert to json
    rGeo_response = requests.get(r_geo, params=params2).json()

    #Locate keys for variables
    rGeo_response["results"][0]["address_components"]


    for addressComp in rGeo_response["results"][0]["address_components"]:
        if addressComp["types"][0] == "country":
            nat = addressComp["long_name"]
            
    return nat
#add country name to "countries" list.
for index, row in hotel_df.iterrows():
    countries.append(pullCountryforcoord(row.Latitude, row.Longitude))   

In [7]:
#Add country column to hotel_df
hotel_df['Country'] = countries
hotel_df

,City,Latitude,Longitude,Humidity,Pressure,Wind Speed (Mph),Cloud Cover (%),Temperature (F),Hotel Name,Hotel Address,Hotel Rating,Country
0,san patricio,19.22,-104.70,78,1014,3.69,0,71.01,,,,Mexico
1,pochutla,15.74,-96.47,62,1011,3.36,0,82.99,,,,Mexico
2,ixtapa,20.70,-105.20,61,1014,1.48,0,71.87,,,,Mexico
3,la reforma,25.08,-108.05,83,1007,2.84,0,73.94,,,,Mexico
4,harper,4.38,-7.72,83,1012,2.30,0,79.81,,,,Liberia
5,namibe,-15.20,12.15,60,1012,3.02,0,76.91,,,,Angola
6,arandis,-22.42,14.97,23,1016,8.21,0,71.62,,,,Namibia
7,paita,-5.09,-81.11,76,1014,7.61,0,71.69,,,,Peru
8,maumere,-8.62,122.21,63,1009,1.70,0,84.90,,,,Indonesia


In [8]:
# Using the template add the hotel marks to the heatmap
    
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Rating</dt><dd>{Hotel Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Address</dt><dd>{Hotel Address}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations2, info_box_content=hotel_info)
fig2 = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(locations2, weights= hotel_df.Humidity, dissipating=False, max_intensity=100,
                                point_radius=1)

fig2.add_layer(heat_layer)
fig2.add_layer(marker_layer)

# Display figure
fig2

Figure(layout=FigureLayout(height='420px'))